# Tarea 20% de SPSI 2025/2026

**Asignatura:**         Seguridad y Protección de Sistemas Informáticos (SPSI)  
**Grupo de trabajo:**   9  
**Curso:**              2025/2026  
**Alumnos:**             Martín Hernández Ruiz,
                         María Megías Moyano,
                         Patricia Cobos Rueda

---

## Índice

1. [Introducción teórica al cifrado de Vigenère](#intro-vigenere)  
2. [Implementación en Python del criptosistema de Vigenère](#implementacion)  
   2.1. [Normalización del texto y representación interna](#normalizacion)  
   2.2. [Funciones de cifrado y descifrado](#funciones-vigenere)  
   2.3. [Pruebas básicas](#pruebas-basicas)  
3. [Laboratorio de criptoanálisis del Vigenère](#laboratorio)  
   3.1. [Índice de coincidencia e índice de Friedman](#ic-friedman)  
   3.2. [Análisis de Kasiski](#kasiski)  
   3.3. [Ruptura de las columnas como cifrados de César](#ruptura-cesar)  
4. [Aplicación al criptograma propuesto en las tareas de SPSI](#aplicacion-criptograma)  
5. [Conclusiones](#conclusiones)  

---


## 1. Introducción teórica al cifrado de Vigenère <a id="intro-vigenere"></a>

El **cifrado de Vigenère** es un ejemplo clásico de **cifrado por sustitución polialfabética**.  
En lugar de usar un único alfabeto de sustitución (como en el cifrado de César), utiliza varios alfabetos distintos, controlados por una **clave** repetida periódicamente. Esto tiene como objetivo principal **disfrazar las frecuencias de las letras del texto llano** y dificultar el análisis de frecuencia directo.   

Históricamente, el cifrado fue atribuido a Blaise de Vigenère, aunque la idea aparece ya en trabajos anteriores de Belasso y Trithemius. 

Durante siglos se consideró prácticamente irrompible; llegó a ser conocido como **le chiffre indéchiffrable** y a principios del siglo XX aún se afirmaba que era “imposible de romper”. Sin embargo, su reputación era inmerecida: el examen de Kasiski y, más tarde, la prueba de Friedman mostraron que es vulnerable si se analiza con las herramientas adecuadas.   

### 1.1. Definición formal básica

Sea n ∈ w* A un alfabeto de n símbolos. Una clave de Vigenère es cualquier elemento K de ex p(A)*. La función de cifrado sobre el alfabeto A de n símbolos según la clave K es E<sub>k</sub> : exp(A)* → exp(A)* definida como sigue:

  E<sub>k</sub>(s)= {(f<sup>-1</sup>)((f(s<sub>j</sub>) + f (K<sub>j</sub><sup>len(s)</sup> )) mod n)}<sub>j</sub>

donde s = ⟨s<sub>j</sub> ⟩<sub>j</sub>, K<sup>len(s)</sup> es la yuxtaposición de la expresión K consigo misma y eventual truncamiento final hasta conseguir así una expresión con la misma longitud que s. La función de descifrado sobre ese alfabeto y clave es D<sub>K</sub> : exp(A)∗ → exp(A)∗ definida por:  

  D<sub>k</sub>(s)= {(f<sup>-1</sup>)((f(s<sub>j</sub>) - f (K<sub>j</sub><sup>len(s)</sup> )) mod n)}<sub>j</sub>

En notación compacta (como en las transparencias), la función de cifrado \(E_K\) toma una palabra \(s\) y devuelve otra palabra de la misma longitud, sumando “texto + clave” módulo \(n\), mientras que la función de descifrado \(D_K\) resta la clave. Se cumple que \(D_K \circ E_K = \mathrm{id}\), es decir, descifrar lo cifrado recupera exactamente el texto original.   

### 1.2. Idea de seguridad y debilidad principal

La idea intuitiva es que **una misma letra de texto llano no siempre se cifra como la misma letra de texto cifrado**. Por ejemplo, la letra más frecuente del inglés, `E`, puede transformarse en muchas letras distintas en posiciones diferentes del mensaje, dependiendo de la letra de la clave que se esté usando en cada caso. Esto rompe el patrón tan evidente de un cifrado monoalfabético y hace que el análisis de frecuencias simple deje de funcionar. 

Sin embargo, la gran **debilidad** del cifrado de Vigenère está precisamente en la **estructura periódica de la clave**. Si un criptoanalista consigue averiguar la **longitud de la clave** \(m\), el texto cifrado puede reorganizarse en \(m\) columnas, cada una de las cuales se comporta como un **cifrado de César independiente**. Entonces es posible aplicar técnicas de análisis de frecuencias (χ², índice de coincidencia, etc.) columna a columna y recuperar la clave completa.   

Los dos métodos clásicos que explotan esta debilidad son:

- El **examen de Kasiski**, que busca repeticiones de bloques en el criptograma y utiliza las distancias entre ellas para proponer candidatos a longitud de clave.   
- La **prueba de Friedman** (o prueba kappa), que se basa en el **índice de coincidencia** y permite estimar la longitud de la clave a partir de las frecuencias globales (o por columnas) del texto cifrado.   

En el resto del cuaderno implementaremos tanto el cifrado y descifrado de Vigenère como estas técnicas de criptoanálisis, siguiendo el enfoque desarrollado en las transparencias de teoría y los textos de Hoffstein–Pipher–Silverman y Dalkilic–Güngör.

---


## 2. Implementación en Python del criptosistema de Vigenère <a id="implementacion"></a>

### 2.1. Normalización del texto y representación interna <a id="normalizacion"></a>

Para cumplir el modelo matemático visto en las transparencias de SPSI , debemos garantizar que el texto llano
y la clave contienen **solo caracteres del alfabeto**. Por eso aplicamos:

- conversión a **mayúsculas**,  
- eliminación de caracteres no alfabéticos,  
- mapeo letra ↔ número.

Esto coincide con las prácticas habituales de los cifrados clásicos y con la forma en que se
presenta el algoritmo en el material de teoría.


In [15]:
import string

# Alfabeto que usaremos: letras mayúsculas A-Z
ALPHABET = string.ascii_uppercase               # LLamamos a la función de string de python que contiene todas las letras del abecedario
ALPHABET_SIZE = len(ALPHABET)


def normalize_text(text: str) -> str:           # Normaliza texto: convierte a mayúsculas y elimina caracteres que no sean A-Z
    text = text.upper()                         # Convierte todo el texto a mayúsculas
    normalized = []                             # Lista donde acumularemos solo los caracteres A-Z
    for ch in text:                             # Recorre cada carácter del texto en mayúsculas
        if ch in ALPHABET:                      # Comprueba si el carácter pertenece al alfabeto A-Z
            normalized.append(ch)               # Añade la letra válida a la lista de normalizados
    return "".join(normalized)                  # Devuelve la cadena resultante formada solo por letras A-Z


def char_to_int(ch: str) -> int:                # Convierte un carácter 'A'-'Z' a entero 0-25
    return ord(ch) - ord('A')                   # Resta el código Unicode de 'A' para mapear 'A'->0, 'B'->1, ... 'Z'->25


def int_to_char(n: int) -> str:                 # Convierte un entero 0-25 a carácter 'A'-'Z' (con módulo 26)
    n = n % ALPHABET_SIZE                       # Normaliza el entero aplicando módulo ALPHABET_SIZE para asegurar 0-25
    return chr(n + ord('A'))                    # Suma el código de 'A' y convierte el entero a carácter correspondiente


def normalize_key(key: str) -> str:             # Normaliza la clave (solo letras A-Z, mayúscula) y valida no vacía
    k = normalize_text(key)                     # Llama a normalize_text para obtener la clave en A-Z sin caracteres no permitidos
    if not k:                                    # Comprueba si la clave normalizada está vacía
        raise ValueError("La clave debe " \
        "contener " \
        "al menos una letra A-Z.")               # Lanza excepción si no hay letras válidas
    return k                                     # Devuelve la clave normalizada y validada

### 2.2. Funciones de cifrado y descifrado <a id="funciones-vigenere"></a>

Implementamos ahora las funciones principales del criptosistema de Vigenère:
- `vigenere_encrypt(plaintext, key)`
- `vigenere_decrypt(ciphertext, key)`


In [16]:
def vigenere_encrypt(text: str, key: str) -> str:       # Define la función de cifrado Vigenère
    P = normalize_text(text)                            # Normaliza el texto de entrada (solo A-Z, mayúsculas)
    K = normalize_key(key)                              # Normaliza y valida la clave (no vacía, A-Z)
    cadena_cifrar = []                                  # Lista donde acumularemos los caracteres cifrados
    key_len = len(K)                                    # Longitud de la clave normalizada

    for i, ch in enumerate(P):                          # Recorre cada carácter del texto normalizado con su índice
        letra_val = char_to_int(ch)                     # Convierte la letra del texto a su valor numérico 0-25
        key_val = char_to_int(K[i % key_len])           # Toma la letra de la clave correspondiente (repetida) y la convierte a número
        cif_val = (letra_val + key_val) % ALPHABET_SIZE # Suma texto+clave módulo tamaño del alfabeto para cifrar
        cif_char = int_to_char(cif_val)                 # Convierte el valor numérico cifrado de vuelta a letra
        cadena_cifrar.append(cif_char)                  # Añade la letra cifrada a la lista de salida

    texto_cifrado = "texto_cifrado.txt"                 # Nombre del archivo donde se guardará el texto cifrado    
    with open(texto_cifrado, 'w+') as archivo:          # Abre (o crea) el archivo de texto cifrado en modo escritura
        archivo.write("".join(cadena_cifrar))           # Escribe la cadena cifrada en el archivo

    return "".join(cadena_cifrar)                       # Une y devuelve la cadena cifrada final


def vigenere_decrypt(cadena_cif: str, key: str) -> str: # Define la función de descifrado Vigenère
    C = normalize_text(cadena_cif)                      # Normaliza el texto cifrado (solo A-Z, mayúsculas)
    K = normalize_key(key)                              # Normaliza y valida la clave usada para descifrar
    descif_chars = []                                   # Lista donde acumularemos los caracteres descifrados
    key_len = len(K)                                    # Longitud de la clave normalizada

    for i, ch in enumerate(C):                          # Recorre cada carácter del texto cifrado con su índice
        char_val = char_to_int(ch)                      # Convierte la letra cifrada a su valor numérico 0-25
        key_val = char_to_int(K[i % key_len])           # Toma la letra de la clave correspondiente y la convierte a número
        descif_val = (char_val-key_val) % ALPHABET_SIZE # Resta clave al cifrado módulo alfabeto para obtener el valor del texto llano
        descif_char = int_to_char(descif_val)           # Convierte el valor numérico descifrado de vuelta a letra
        descif_chars.append(descif_char)                # Añade la letra descifrada a la lista de salida
    return "".join(descif_chars)                        # Une y devuelve la cadena del texto llano resultante

### 2.3. Pruebas básicas <a id="pruebas-basicas"></a>

Comprobamos ahora que cifrar y luego descifrar un mensaje devuelve el texto original (normalizado).


In [17]:
from collections import Counter

while True:
	print("|--------------|")
	print("|  1 Cifrar    |")
	print("|  2 Descifrar |")
	print("|  3 Salir     |")
	print("|--------------|")
	try:
		opt = int(input("Ingrese opcion: "))
	except ValueError:
		print("Opción inválida, introduzca un número (1-3).")
		continue

	# -----------------------------------------------------------
	# 1) CIFRAR Y GUARDAR EN ARCHIVO
	# -----------------------------------------------------------
	if opt == 1:
		ruta_entrada = "texto.txt"
		ruta_clave = "clave.txt"
		ruta_salida = "texto_cifrado.txt"

		# Leer texto plano
		try:
			with open(ruta_entrada, "r", encoding="utf-8") as f:
				texto_plano = f.read()
		except FileNotFoundError:
			print(f"No se encontró el archivo {ruta_entrada}")
			continue


		try:
			with open(ruta_clave, "r", encoding="utf-8") as f:
				clave = f.read().strip()								#Leemos la clave y la guardamos
		except FileNotFoundError:
			print(f"No se encontró el archivo {ruta_clave}")
			continue

		# Cifrar
		texto_cifrado = vigenere_encrypt(texto_plano, clave)

		# Guardar salida
		with open(ruta_salida, "w", encoding="utf-8") as f:
			f.write(texto_cifrado)

		print(f"\nTexto cifrado guardado en '{ruta_salida}'\n")

	# -----------------------------------------------------------
	# 2) DESCIFRAR DESDE ARCHIVO Y GUARDAR SALIDA
	# -----------------------------------------------------------
	elif opt == 2:
		ruta_cifrado = "texto_cifrado.txt"
		ruta_clave = "clave.txt"
		ruta_salida = "texto_descifrado.txt"

		# Leer texto cifrado
		try:
			with open(ruta_cifrado, "r", encoding="utf-8") as f:
				texto_cifrado = f.read()
		except FileNotFoundError:
			print(f"No se encontró el archivo {ruta_cifrado}")
			continue

		# Leer clave
		try:
			with open(ruta_clave, "r", encoding="utf-8") as f:
				clave = f.read().strip()
				
		except FileNotFoundError:
			print(f"No se encontró el archivo {ruta_clave}")
			continue

		# Descifrar
		texto_descifrado = vigenere_decrypt(texto_cifrado, clave)

		# Guardar salida
		with open(ruta_salida, "w", encoding="utf-8") as f:
			f.write(texto_descifrado)

		print(f"\nTexto descifrado guardado en '{ruta_salida}'\n")

	# -----------------------------------------------------------
	# 3) SALIR
	# -----------------------------------------------------------
	elif opt == 3:
		print("Saliendo...")
		break

	else:
		print("Opción no válida.\n")


|--------------|
|  1 Cifrar    |
|  2 Descifrar |
|  3 Salir     |
|--------------|

Texto cifrado guardado en 'texto_cifrado.txt'

|--------------|
|  1 Cifrar    |
|  2 Descifrar |
|  3 Salir     |
|--------------|

Texto descifrado guardado en 'texto_descifrado.txt'

|--------------|
|  1 Cifrar    |
|  2 Descifrar |
|  3 Salir     |
|--------------|
Saliendo...


Vemos como se ha cifrado y descifrado el texto deseado correctamente, <b> CONOCIENDO </b> la clave de cifrado.

¿Pero qué pasaría si <b> NO CONOCEMOS </b> la clave de cifrado?

¿Seriamos capaces de descifrar el texto cifrado por el algorítmo de Vigenère?.

Eso será lo que estudiaremos en los siguienets apartados del cuaderno.

---

## 3. Laboratorio de criptoanálisis del Vigenère <a id="laboratorio"></a>




### 3.1. Índice de coincidencia e índice de Friedman <a id="ic-friedman"></a>

El **índice de coincidencia (IC)** de un texto se define como:

  IC=   ∑<sub>i</sub> f<sub>i</sub> (f<sub>i</sub> − 1 ) / N ( N − 1 )
	​


donde:
- f<sub>i</sub> es la frecuencia absoluta de la i-ésima letra,
- N es el número total de letras.

El IC lo que mide con la fórmula indicada arriba cómo de parecida es la distribución de las frecuencias de un texto a la del idioma natural.
Si analizamos los valores tipicos del IC observamos que:

  | Tipo de texto                    | IC aproximado                    |
  | -------------------------------- | -------------------------------- |
  | **Inglés natural**               | ~0.066                           |
  | **Español**                      | ~0.07                            |
  | **Texto aleatorio**              | ~0.038                           |
  | **Vigenère con clave muy larga** | cercano a 0.038                  |
  | **Vigenère con clave corta**     | entre 0.04 y 0.06 según longitud |

Cuanto más bajo son los valores del IC, más aleatorio parece el texto.
El español tiene un mayor IC puesto que la concentración de letras que se repiten es mayor, es decir la distribución de frecuencias es más desigual.
  - Alta frecuencia de: E, A, O, S, N, R, L
  - Menor diversidad en las consonantes menos comunes.

El cifrado Vigenère mezcla varios alfabetos según la longitud de clave m.
Friedman lo que propon es:

  - Si la clave es corta → IC sube
  - Si la clave es larga → IC baja

Por eso, IC te permite estimar la longitud de la clave.
La fórmula de Friedman para estimar 'm' es:

  m ≈ ( K<sub>0</sub> - K<sub>r</sub> ) N / ( N - 1 ) IC - K<sub>0</sub> + K<sub>r</sub>


Para texto en **inglés** no cifrado, el IC típico es aproximadamente 0.066,
mientras que para texto completamente aleatorio es de alrededor de 0.038.

En un Vigenère con clave de longitud "m", cada columna se comporta como un cifrado
de César sobre texto en claro, por lo que su IC se aproxima al de un idioma natural.

El **método de Friedman** usa el IC global del criptograma para estimar la longitud
de la clave.


In [18]:

# Índices de coincidencia típicos por idioma
IC_IDIOMA = {
    "en": 0.066,   # Inglés natural
    "es": 0.077    # Español natural
}

# Índice de coincidencia (IC)
# Fórmula: IC = sum( f_i * (f_i - 1) ) / ( N * (N - 1) )
def indice_coincidencia(texto: str) -> float:
    texto_normalizado = normalize_text(texto)                       # Normaliza el texto (solo letras A-Z, mayúsculas)
    longitud = len(texto_normalizado)                               # Longitud del texto normalizado

    if longitud < 2:                                                # Si la longitud es menor que 2, no se puede calcular el IC  
        return 0.0

    frecuencias = Counter(texto_normalizado)                        # Cuenta las frecuencias de cada letra en el texto normalizado

    numerador = sum(f * (f - 1) for f in frecuencias.values())      # Numerador: sumatorio de f_i * (f_i - 1)
    denominador = longitud * (longitud - 1)                         # Denominador: N * (N - 1)

    return numerador / denominador                                  # Devuelve el índice de coincidencia calculado

# Estimación de longitud de clave usando la fórmula de Friedman
# Fórmula completa = m ≈ (K0 - Kr) * N / ( N * (IC - Kr) + (K0 - IC) )
def estimar_longitud_clave_friedman(texto: str, idioma: str = "es", tam_alfabeto: int = 26) -> float:
    if idioma not in IC_IDIOMA:                                     # Verifica que el idioma es soportado
        raise ValueError("Idioma no soportado. Usa 'es' o 'en'.")   # Lanza excepción si el idioma no es válido

    K0 = IC_IDIOMA[idioma]                                          # IC típico del idioma natural

    Kr = 1 / tam_alfabeto                                           # IC esperado para texto aleatorio

    texto_normalizado = normalize_text(texto)                       # Normaliza el texto (solo letras A-Z, mayúsculas)
    longitud = len(texto_normalizado)                               # Longitud del texto normalizado

    if longitud < 2:                                                # Si la longitud es menor que 2, no se puede estimar la longitud de clave
        return 0.0

    IC = indice_coincidencia(texto_normalizado)                     # Calcula el IC del texto normalizado

    denominador = longitud * (IC - Kr) + (K0 - IC)                  # Denominador de la fórmula

    if denominador <= 0:                                            # Caso en el que la fórmula no es fiable
        return 0.0

    numerador = (K0 - Kr) * longitud                                # Numerador

    longitud_clave = numerador / denominador                        # Estimación final de la longitud de clave

    return longitud_clave                                           # Devuelve la estimación de la longitud de clave

Supongamos:

texto_plano.txt → texto sin cifrar (en español)

texto_cifrado.txt → mismo texto cifrado con Vigenère (clave CAPITAN)

In [19]:
# Ruta a tus ficheros 
RUTA_TEXTO_PLANO = "texto.txt"
RUTA_TEXTO_CIFRADO = "texto_cifrado.txt"

# Leer los textos desde los archivos
with open(RUTA_TEXTO_PLANO, "r", encoding="utf-8") as archivo:   # Abre el archivo de texto plano en modo lectura
    texto_plano = archivo.read()                                 # Lee todo el contenido del archivo y lo almacena en la variable texto_palano

with open(RUTA_TEXTO_CIFRADO, "r", encoding="utf-8") as archivo: # Abre el archivo de texto cifrado en modo lectura
    texto_cifrado = archivo.read()                               # Leer todo el contenido y guardarlo en texto_cifrado

print("Primeros caracteres del texto en claro:")                 # Imprime los primeros 200 caracteres del texto plano
print(texto_plano[:200])  

print("\nPrimeros caracteres del texto cifrado:")                # Imprime los primeros 200 caracteres del texto cifrado
print(texto_cifrado[:200])


Primeros caracteres del texto en claro:
En un lugar de la Mancha, de cuyo nombre no quiero acordarme,
no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero,
adarga antigua, rocín flaco y galgo corredor.
Una olla de algo más v

Primeros caracteres del texto cifrado:
QVAHPFJETHVGAZTRUTIJYGFBSPSDWRRGSIGQKLSLFSTHRMMRGSZMUAWLZWMGQGJQHXZAHIAHLTGENKFYEYHWVQTGHDLHRCWKDLYXVGMLGLKLDRVMXPAEHGFRTGWSJJENKFXOEKIVAZAHEZOPCHVVLTHQKHIIUUFHGCVEZRBLEDBQIHPLVQURFXHRLHMQTUMCBXIDVRIT


**Comprobar que la clave CAPITAN cifra correctamente**

In [20]:
# Normalizar los textos
texto_plano_normalizado = normalize_text(texto_plano)                   # Normaliza el texto original
texto_cifrado_normalizado = normalize_text(texto_cifrado)               # Normaliza el texto cifrado del archivo

cifrado_generado = vigenere_encrypt(texto_plano_normalizado, clave)     # Cifra el texto plano normalizado con la clave dada

print("Cifrado generado (primeros 200 caracteres):")                    # Imprime los primeros 200 caracteres del cifrado generado
print(cifrado_generado[:200])

print("\nCifrado del fichero (primeros 200 caracteres):")               # Imprime los primeros 200 caracteres del texto cifrado normalizado
print(texto_cifrado_normalizado[:200])

# Verificar si coinciden los primeros 200 caracteres
print("\n¿Coinciden los primeros 200 caracteres?", cifrado_generado[:200] == texto_cifrado_normalizado[:200])


Cifrado generado (primeros 200 caracteres):
QVAHPFJETHVGAZTRUTIJYGFBSPSDWRRGSIGQKLSLFSTHRMMRGSZMUAWLZWMGQGJQHXZAHIAHLTGENKFYEYHWVQTGHDLHRCWKDLYXVGMLGLKLDRVMXPAEHGFRTGWSJJENKFXOEKIVAZAHEZOPCHVVLTHQKHIIUUFHGCVEZRBLEDBQIHPLVQURFXHRLHMQTUMCBXIDVRIT

Cifrado del fichero (primeros 200 caracteres):
QVAHPFJETHVGAZTRUTIJYGFBSPSDWRRGSIGQKLSLFSTHRMMRGSZMUAWLZWMGQGJQHXZAHIAHLTGENKFYEYHWVQTGHDLHRCWKDLYXVGMLGLKLDRVMXPAEHGFRTGWSJJENKFXOEKIVAZAHEZOPCHVVLTHQKHIIUUFHGCVEZRBLEDBQIHPLVQURFXHRLHMQTUMCBXIDVRIT

¿Coinciden los primeros 200 caracteres? True


**Demostración teórica: IC del texto sin cifrar vs cifrado**

Ahora vamos a calcular el Índice de Coincidencia de ambos:

In [21]:
# Calcular el índice de coincidencia (IC) de los textos
IC_texto_plano = indice_coincidencia(texto_plano)                          # Calcula el IC del texto plano   
IC_texto_cifrado = indice_coincidencia(texto_cifrado)                      # Calcula el IC del texto cifrado

# Mostrar longitudes de los textos
print("Longitud del texto en claro:", len(normalize_text(texto_plano)))     # Imprime la longitud del texto plano normalizado
print("Longitud del texto cifrado  :", len(normalize_text(texto_cifrado)))  # Imprime la longitud del texto cifrado normalizado
print() 

# Mostrar los índices de coincidencia
print("IC(texto en claro)  =", IC_texto_plano)                              # Imprime el IC del texto plano
print("IC(texto cifrado)   =", IC_texto_cifrado)                            # Imprime el IC del texto cifrado
print("IC español teórico  ≈", IC_IDIOMA["es"])                             # Imprime el IC teórico del español
print("IC aleatorio (1/26) ≈", 1/26)                                        # Imprime el IC esperado para texto aleatorio



Longitud del texto en claro: 2250
Longitud del texto cifrado  : 2250

IC(texto en claro)  = 0.07474689985672645
IC(texto cifrado)   = 0.041259226322810136
IC español teórico  ≈ 0.077
IC aleatorio (1/26) ≈ 0.038461538461538464


Con esto se demuestra:
  - IC del texto en claro está cerca del IC del idioma (≈ 0.077 en español).
  - IC del texto cifrado con Vigenère está más cerca del IC aleatorio (≈ 0.038), aunque algo más alto por ser clave de longitud 7.

**Demostración del método de Friedman**

Aplicamos Friedman al texto en claro y al texto cifrado:

**SEGUIR EDITANDO POR AQUI**


In [ ]:
m_plain_es = estimar_longitud_clave_friedman(texto_plano, idioma="es")
m_cipher_es = estimar_longitud_clave_friedman(texto_cifrado, idioma="es")
m_cipher_en = estimar_longitud_clave_friedman(texto_cifrado, idioma="en")

print("=== Friedman sobre texto en claro (español) ===")
print("IC(plain) =", IC_texto_plano)
print("Estimación m (español) =", m_plain_es)

print("\n=== Friedman sobre texto cifrado (español) ===")
print("IC(cipher) =", IC_texto_cifrado)
print("Estimación m (español) =", m_cipher_es)

print("\n=== Friedman sobre texto cifrado (inglés) ===")
print("IC(cipher) =", IC_texto_cifrado)
print("Estimación m (inglés) =", m_cipher_en)


=== Friedman sobre texto en claro (español) ===
IC(plain) = 0.07474689985672645
Estimación m (español) = 1.0620646048081186

=== Friedman sobre texto cifrado (español) ===
IC(cipher) = 0.041259226322810136
Estimación m (español) = 13.697340121754154

=== Friedman sobre texto cifrado (inglés) ===
IC(cipher) = 0.041259226322810136
Estimación m (inglés) = 9.804756919930876


AÑADIR ANALISIS DE RESULTADOS

### 3.2. Análisis de Kasiski <a id="kasiski"></a>

El **método de Kasiski** se basa en:

1. Buscar **repeticiones** de secuencias de longitud 3 o más en el criptograma.
2. Calcular las distancias (en número de caracteres) entre apariciones consecutivas.
3. Calcular el **MCD** (máximo común divisor) y factores de esas distancias.
4. Los divisores más frecuentes son buenos candidatos a longitud de clave.

Implementamos funciones para:

- Encontrar repeticiones y distancias.
- Listar factores de las distancias.


In [14]:
from collections import defaultdict, Counter                                        # defaultdict: dict con valor por defecto; Counter: contador de frecuencias
from math import gcd                                                                # gcd = máximo común divisor (ahora mismo NO se usa en tu código)

def normalizar_texto(texto: str) -> str:                                            # Convierte a mayúsculas y se queda solo con letras A-Z (típico en Vigenère/Kasiski)
    texto = texto.upper()                                                           # Pasa todo a mayúsculas
    return "".join(c for c in texto if "A" <= c <= "Z")                             # Filtra solo letras (quita espacios, tildes, signos, etc.)

def kasiski_distancias(texto: str, longitud_min: int = 3, longitud_max: int = 6):
   
    texto_limpio = normalizar_texto(texto)                                          # Normaliza el texto para trabajar solo con A-Z
    posiciones_por_secuencia = defaultdict(list)                                    # Diccionario: secuencia -> lista de posiciones donde aparece

    for n in range(longitud_min, longitud_max + 1):                                 # Recorremos tamaños de subcadena desde longitud_min hasta longitud_max Para cada longitud n
        for i in range(len(texto_limpio) - n + 1):                                  # Para cada posición inicial posible
            secuencia = texto_limpio[i:i+n]                                         # Extrae la subcadena de longitud n
            posiciones_por_secuencia[secuencia].append(i)                           # Guarda la posición donde aparece esa secuencia

    distancias_por_secuencia = {}                                                   # Aquí guardaremos: secuencia -> [distancias]
    for secuencia, lista_posiciones in posiciones_por_secuencia.items():            # Recorremos cada secuencia y sus posiciones
        if len(lista_posiciones) >= 2:                                              # Solo interesa si aparece al menos 2 veces
            distancias = []                                                         # Lista de distancias entre apariciones consecutivas
            for j in range(1, len(lista_posiciones)):                               # Desde la segunda aparición hasta la última
                distancias.append(lista_posiciones[j] - lista_posiciones[j-1])      # Resta posiciones consecutivas
            if distancias:                                                          # Si hay alguna distancia calculada
                distancias_por_secuencia[secuencia] = distancias                    # Guarda la lista de distancias para esa secuencia

    return distancias_por_secuencia                                                 # Devuelve el diccionario final

def factorizar(n: int):
    
    return [d for d in range(2, n+1) if n % d == 0]                                 # Lista todos los divisores d (>=2) que dividen a n

def kasiski_estadisticas_factores(texto: str, longitud_min: int = 3, 
                                  longitud_max: int = 6):
    
    distancias_por_secuencia = kasiski_distancias(texto, longitud_min=longitud_min, 
                                                  longitud_max=longitud_max)        # Distancias
    conteo_factores = Counter()                                                     # Contador de factores (frecuencias)

    for secuencia, lista_distancias in distancias_por_secuencia.items():            # Para cada secuencia repetida
        for distancia in lista_distancias:                                          # Para cada distancia encontrada
            for factor in factorizar(distancia):                                    # Factoriza la distancia
                conteo_factores[factor] += 1                                        # Incrementa cuántas veces aparece ese factor

    return distancias_por_secuencia, conteo_factores                                # Devuelve distancias y conteos

def kasiski_mejor_longitud_clave(texto_cifrado: str, longitud_min: int = 3, 
                                 longitud_max: int = 6):
    
    _, conteo_factores = kasiski_estadisticas_factores(texto_cifrado, 
                                                       longitud_min=longitud_min, 
                                                       longitud_max=longitud_max)  # Ejecuta Kasiski

    # Filtramos factores: descartamos los triviales y los demasiado grandes
    # (en textos pequeños, factores grandes suelen ser ruido)

    factores_filtrados = {factor: cuenta for factor, 
                          cuenta in conteo_factores.items() if 2 <= factor <= 30}   # Filtrado
    if not factores_filtrados:                                                      # Si no queda ningún candidato
        return None, conteo_factores                                                # No se puede estimar longitud de clave

    # Elegimos el factor con mayor frecuencia como mejor candidato de longitud de clave

    mejor_m = max(factores_filtrados, key=factores_filtrados.get)                   # factor con mayor conteo
    return mejor_m, conteo_factores                                                 # Devuelve la mejor longitud y el conteo completo


In [13]:
distances, factor_counts = kasiski_estadisticas_factores(               # Llama a la función Kasiski que calcula distancias y cuenta factores
    texto_cifrado_normalizado,                                          # Texto cifrado ya normalizado (solo A-Z, sin espacios/signos)
    longitud_min=3,                                                     # Longitud mínima de las secuencias repetidas a buscar (trigramas)
    longitud_max=6                                                      # Longitud máxima de las secuencias repetidas a buscar
)

print("Algunas secuencias repetidas y sus distancias:")                 # Muestra un título informativo por pantalla

for seq, ds in list(distances.items())[:10]:                            # Recorre como mucho las 10 primeras entradas del diccionario distances
    print(f"{seq} -> {ds}")                                             # Imprime la secuencia (seq) y su lista de distancias (ds)

print("\nFactores más frecuentes (candidatos a longitud de clave):")    # Salto de línea + título para el ranking de factores

for factor, count in factor_counts.most_common(15):                     # Recorre los 15 factores más repetidos (más probables como long. de clave)
    print(f"{factor} -> {count}")                                       # Imprime el factor y cuántas veces aparece

m_est, factor_counts = kasiski_mejor_longitud_clave(                    # Calcula la longitud de clave más probable según el factor más frecuente
    texto_cifrado_normalizado                                           # Se aplica sobre el mismo texto cifrado normalizado
)                                                                       # Devuelve (m_est, factor_counts); factor_counts se recalcula internamente

print("Factores detectados por Kasiski (ordenados):")                   # Título para volver a imprimir los factores ordenados por frecuencia

for f, c in factor_counts.most_common(15):                              # Recorre otra vez los 15 factores más frecuentes
    print(f"{f} -> {c}")                                                # Imprime factor (f) y su frecuencia (c)

print("\nLongitud de clave estimada por Kasiski:", m_est)               # Muestra el resultado final: longitud de clave estimada



Algunas secuencias repetidas y sus distancias:
QVA -> [578]
JET -> [1549]
THV -> [567]
HVG -> [745]
GAZ -> [1109]
TRU -> [1500]
TIJ -> [280, 1089]
IJY -> [263, 332, 1000]
JYG -> [595]
PSD -> [1350]

Factores más frecuentes (candidatos a longitud de clave):
17 -> 260
2 -> 199
3 -> 148
34 -> 122
4 -> 96
51 -> 91
5 -> 89
7 -> 68
68 -> 61
85 -> 51
6 -> 49
8 -> 46
13 -> 41
119 -> 40
14 -> 39
Factores detectados por Kasiski (ordenados):
17 -> 260
2 -> 199
3 -> 148
34 -> 122
4 -> 96
51 -> 91
5 -> 89
7 -> 68
68 -> 61
85 -> 51
6 -> 49
8 -> 46
13 -> 41
119 -> 40
14 -> 39

Longitud de clave estimada por Kasiski: 17


### Aplicación del método de Kasiski al criptograma

Además del método de Friedman, aplicamos el método de Kasiski para estimar la longitud de la clave.
Hemos implementamos una búsqueda de secuencias repetidas de longitud entre 3 y 6
caracteres en el criptograma, y calculamos las distancias entre apariciones consecutivas de cada una
de dichas secuencias. A continuación factorizamos esas distancias y contamos qué factores aparecen
con mayor frecuencia.

De este modo, la presencia reiterada de un mismo factor, así como de sus múltiplos, indica que una parte 
significativa de las repeticiones del criptograma está alineada con dicha periodicidad, 
lo que refuerza su plausibilidad como longitud de clave.

Por tanto, mientras que el método de Friedman sitúa la longitud de la clave en un rango aproximado, el método de Kasiski 
permite refinar esta estimación y concluye
que la longitud de la clave más plausible es:

Para nuestro ejemplo, el método de Friedman nos dio un aproximación inical de 13. Esta estimación de longitud esta basada en 
propiedades estadisticas globales de texto dando una longitud de clave posible que, aunque quiere aproximarse, dista en cuanto a 
la estimación realizada por el metodo Kasiski; que nos proporciona una estimación de 17 para nuestra clave.  Estimación mucho
más exacta, que la proporcionada por Friedman.

\[
m = 17
\]


### 3.3. Ruptura de las columnas como cifrados de César <a id="ruptura-cesar"></a>

Una vez tenemos un candidato para la longitud de la clave \(m\):

1. Dividimos el criptograma en \(m\) **columnas**, donde la columna \(j\)
   contiene los caracteres \(C_i\) tales que \(i \equiv j \pmod m\).
2. Cada columna se comporta como un **cifrado de César** aplicado sobre texto en claro.
3. Para cada desplazamiento posible (0–25) calculamos una medida de ajuste con las
   frecuencias típicas del idioma (por ejemplo, un \(\chi^2\)).
4. Elegimos el desplazamiento que mejor se ajusta a las frecuencias esperadas.
5. Traducimos ese desplazamiento a una letra de clave.
Idea teórica: por qué cada columna es un César

Sabemos que el Vigenère hace:

𝐶<sub>𝑖</suub> = ( 𝑃<sub>𝑖</sub> + 𝐾<sub>𝑖 mod m</sub>  ) mod 26

Si fijamos una posición j de la clave (0 ≤ j < m), miramos todos los caracteres del criptograma que cumplen:

𝑖 ≡ 𝑗 ( mod 𝑚 )

En esos índices, siempre se ha usado la misma letra de la clave 𝐾 <sub>𝑗</sub>.
Por tanto, la relación entre texto en claro y cifrado en esa columna es:

𝐶<sub>𝑡</sub><sup>(𝑗)</sup> = ( 𝑃<sub>𝑡</sub> <sup>(𝑗)</sup> + 𝐾<sub>𝑗</sub> ) mod 26

Eso es exactamente un cifrado de César con desplazamiento fijo 𝐾<sub>𝑗</sub>.

Conclusión:

Cada columna = César independiente.

Si averiguo el mejor desplazamiento para esa columna → obtengo la letra 𝐾<sub>𝑗</sub>de la clave.
Implementamos ahora estas herramientas.


Paso 1: dividir el criptograma en columnas

In [ ]:
def split_into_columns(text: str, key_len: int):
    """
    Divide el texto normalizado en 'key_len' columnas según posición módulo key_len.
    col[j] contiene los caracteres C_i tales que i ≡ j (mod key_len).
    """
    T = normalize_text(text)
    cols = ["" for _ in range(key_len)]
    for i, ch in enumerate(T):
        cols[i % key_len] += ch
    return cols

# Ejemplo con m = 7
m = m_est
columns = split_into_columns(texto_cifrado_normalizado, m)

for idx, col in enumerate(columns):
    print(f"Columna {idx}: longitud = {len(col)}; primeros 30 caracteres: {col[:30]}")


Paso 2: modelo de frecuencias (usaremos español)

In [ ]:
FRECUENCIAS_ESPANOL = {
    'A': 0.1253, 'B': 0.0142, 'C': 0.0468, 'D': 0.0586, 'E': 0.1368,
    'F': 0.0069, 'G': 0.0101, 'H': 0.0070, 'I': 0.0625, 'J': 0.0044,
    'K': 0.0002, 'L': 0.0497, 'M': 0.0315, 'N': 0.0671, 'O': 0.0868,
    'P': 0.0251, 'Q': 0.0088, 'R': 0.0687, 'S': 0.0798, 'T': 0.0463,
    'U': 0.0393, 'V': 0.0090, 'W': 0.0001, 'X': 0.0022, 'Y': 0.0090,
    'Z': 0.0052,
}


Paso 3: χ² para un desplazamiento de César

Para una columna y un desplazamiento shift:

Supongo que la clave tiene ese desplazamiento → descifro la columna con ese shift.

Cuento las letras del resultado → frecuencias observadas.

Calculo:

𝜒2 = ∑<sub>letra</sub> ( O − E )<sup>2</sup> / E

donde O = observada, E = esperada según el modelo del idioma.

El shift con χ² mínimo es el más probable.

In [ ]:
def chi_cuadrado_para_desplazamiento(columna: str, desplazamiento: int, modelo_frecuencias=FRECUENCIAS_ESPANOL) -> float:
    
    N = len(columna)                                                                    # Número total de caracteres de la columna
    if N == 0:                                                                          # Si la columna está vacía, no se puede hacer análisis estadístico
        return float('inf')                                                             # Devuelve infinito para descartar esta opción

                                                                                        # Descifrado tipo César: se resta el desplazamiento a cada letra
    texto_descifrado = []                                                               # Lista donde se irán almacenando las letras descifradas
    for caracter in columna:                                                            # Recorre cada letra cifrada de la columna
        valor_cifrado = char_to_int(caracter)                                           # Convierte la letra en un valor numérico (0–25)
        valor_plano = (valor_cifrado - desplazamiento) % ALPHABET_SIZE                  # Aplica el descifrado con módulo 26
        texto_descifrado.append(int_to_char(valor_plano))                               # Convierte de nuevo a letra y la añade a la lista

    texto_descifrado = "".join(texto_descifrado)                                        # Une la lista de letras en un único string

    frecuencias_observadas = Counter(texto_descifrado)                                  # Cuenta las apariciones reales de cada letra en el texto obtenido

    chi2 = 0.0                                                                          # Inicializa el acumulador del estadístico χ²
    for letra, probabilidad_esperada in modelo_frecuencias.items():                     # Recorre cada letra del modelo lingüístico
        E = probabilidad_esperada * N                                                   # Frecuencia esperada absoluta para esa letra
        O = frecuencias_observadas.get(letra, 0)                                        # Frecuencia observada (0 si no aparece)
        if E > 0:                                                                       # Evita divisiones por cero
            chi2 += (O - E) ** 2 / E                                                    # Añade el término correspondiente al estadístico χ²

    return chi2                                                                         # Devuelve el valor final de χ² (cuanto menor, mejor encaje con el español)


def mejor_desplazamiento_columna(columna: str, modelo_frecuencias=FRECUENCIAS_ESPANOL):
  
    mejor_desplazamiento = None                                                         # Almacenará el desplazamiento óptimo
    mejor_chi2 = float('inf')                                                           # Valor mínimo de χ² encontrado hasta el momento

    for desplazamiento in range(ALPHABET_SIZE):                                         # Prueba todos los desplazamientos posibles (0–25)
        chi2_actual = chi_cuadrado_para_desplazamiento(columna, desplazamiento, 
                                                       modelo_frecuencias)              # Calcula χ²
        if chi2_actual < mejor_chi2:                                                    # Si el ajuste es mejor que el anterior
            mejor_chi2 = chi2_actual                                                    # Actualiza el mejor valor de χ²
            mejor_desplazamiento = desplazamiento                                       # Guarda el desplazamiento correspondiente

    return mejor_desplazamiento, mejor_chi2                                             # Devuelve el desplazamiento óptimo y su χ² asociado


Paso 4: de los shifts a la clave completa

Como hemos dicho:

El shift que descifra la columna → es el valor numérico de la letra de clave correspondiente.

Si el mejor shift para la columna j es s, entonces la letra de la clave es:

𝐾
𝑗
=
𝑓
−
1
(
𝑠
)
K
j
	​

=f
−1
(s)

In [ ]:
def guess_key_for_length(ciphertext: str, key_len: int, freq_model=SPANISH_FREQ):
    """
    A partir de una longitud de clave, rompe las columnas como César
    y reconstruye la clave completa.
    """
    cols = split_into_columns(ciphertext, key_len)
    key_chars = []
    for j, col in enumerate(cols):
        shift, chi2 = best_shift_for_column(col, freq_model)
        key_char = int_to_char(shift)
        key_chars.append(key_char)
        print(f"Columna {j}: mejor shift = {shift} ({key_char}), χ² = {chi2:.2f}")
    return "".join(key_chars)


Paso 5: aplicar con m = 7 a tu criptograma

Ahora ejecutas:

In [ ]:
m = m_est
key_guess = guess_key_for_length(ciphertext_norm, m, SPANISH_FREQ)
print("\nClave aproximada encontrada:", key_guess)


ULTIMO PASO Ver el descifrado completo

Para rematar:

In [ ]:
plaintext_candidate = vigenere_decrypt(ciphertext_norm, key_guess)
print(plaintext_candidate[:1000])


Detalles observados hasta este punto:
Si usamos un aclave muy grande con 26 caracteres:

La clave casi nunca está sincronizada con patrones internos del texto.

Las repeticiones aleatorias tienden a tener distancias que son múltiplos de números pequeños.

y nos encontaremnos con un 'm' que divide muchosnúmeros peuqñeos y aparecera como factor

## 4. Aplicación al criptograma propuesto en las tareas de SPSI <a id="aplicacion-criptograma"></a>

En el enunciado de las tareas de SPSI se proporciona el siguiente criptograma
cifrado mediante un criptosistema de Vigenère. A partir de lo implementado
anteriormente, debemos averiguar:

- La **longitud de la clave**.
- La **clave**.
- El **mensaje en claro**.

En primer lugar, definimos el texto cifrado en una variable de Python y lo normalizamos.


In [ ]:
ciphertext_raw = """UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWO
FFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZIVQWOFROGTT
NNXEVIVIBPDTTGAHVIACLAYKGJIEQHGECMESNNOCTHSGGNVWTQHKBPR
HMVUOYWLIAFIRIGDBOEBQLIGWARQHNLOISQKEPEIDVXXNETPAXNZGDX
WWEYQCTIGONNGJVHSQGEATHSYGSDVVOAQCXLHSPQMDMETRTMDUXTEQQ
JMFAEEAAIMEZREGIMUECICBXRVQRSMENNWTXTNSRNBPZHMRVRDYNECG
SPMEAVTENXKEQKCTTHSPCMQQHSQGTXMFPBGLWQZRBOEIZHQHGRTOBSG
TATTZRNFOSMLEDWESIWDRNAPBFOFHEGIXLFVOGUZLNUSRCRAZGZRTTA
YFEHKHMCQNTZLENPUCKBAYCICUBNRPCXIWEYCSIMFPRUTPLXSYCBGCC
UYCQJMWIEKGTUBRHVATTLEKVACBXQHGPDZEANNTJZTDRNSDTFEVPDXK
TMVNAIQMUQNOHKKOAQMTBKOFSUTUXPRTMXBXNPCLRCEAEOIAWGGVVUS
GIOEWLIQFOZKSPVMEBLOHLXDVCYSMGOPJEFCXMRUIGDXNCCRPMLCEWT
PZMOQQSAWLPHPTDAWEYJOGQSOAVERCTNQQEAVTUGKLJAXMRTGTIEAFW
PTZYIPKESMEAFCGJILSBPLDABNFVRJUXNGQSWIUIGWAAMLDRNNPDXGN
PTTGLUHUOBMXSPQNDKBDBTEECLECGRDPTYBVRDATQHKQJMKEFROCLXN
FKNSCWANNAHXTRGKCJTTRRUEMQZEAEIPAWEYPAJBBLHUEHMVUNFRPVM
EDWEKMHRREOGZBDBROGCGANIUYIBNZQVXTGORUUCUTNBOEIZHEFWNBI
GOZGTGWXNRHERBHPHGSIWXNPQMJVBCNEIDVVOAGLPONAPWYPXKEFKOC
MQTRTIDZBNQKCPLTTNOBXMGLNRRDNNNQKDPLTLNSUTAXMNPTXMGEZKA
EIKAGQ"""
ciphertext = normalize_text(ciphertext_raw)
len(ciphertext), ciphertext[:100]


### 4.1. IC global y estimación de longitud de clave (Friedman)

Calculamos ahora el índice de coincidencia del criptograma y usamos la
fórmula de Friedman para obtener una primera aproximación de la longitud de la clave.


In [ ]:
IC_cipher = index_of_coincidence(ciphertext)
friedman_estimate = friedman_key_length_estimate(ciphertext)

print("Longitud del criptograma:", len(ciphertext))
print("Índice de coincidencia:", IC_cipher)
print("Estimación de longitud de clave (Friedman):", friedman_estimate)


### 4.2. Análisis de Kasiski sobre el criptograma

A continuación aplicamos el análisis de Kasiski para apoyar la estimación de la
longitud de la clave. Observamos los factores que aparecen con mayor frecuencia.


In [ ]:
distances, factor_counts = kasiski_factor_stats(ciphertext, min_len=3)

print("Algunas secuencias repetidas y sus distancias:")
for seq, ds in list(distances.items())[:10]:
    print(seq, "->", ds)

print("\nFactores más frecuentes (candidatos a longitud de clave):")
for factor, count in factor_counts.most_common(15):
    print(factor, "->", count)


### 4.3. Ruptura tentativa para longitudes de clave candidatas

Seleccionamos manualmente algunas longitudes de clave candidatas (por ejemplo, las más
frecuentes según Kasiski y cercanas a la estimación de Friedman) y probamos a adivinar
la clave completa con el análisis por columnas.


In [ ]:
m = kasiski_best_key_length(ciphertext)[0]
print("\nLongitud de clave estimada por Kasiski:", m)
key_guess = guess_key_for_length(ciphertext, m, SPANISH_FREQ)
print("\nClave aproximada encontrada:", key_guess)

### 4.4. Descifrado con la clave candidata

Una vez tengamos una clave candidata plausible (por ejemplo, aquella que produzca
un texto en claro con apariencia de idioma natural), realizamos el descifrado completo.
En esta celda, el alumno puede fijar manualmente la longitud de clave y la clave
que considere más razonable tras los análisis anteriores.


In [ ]:
plaintext_candidate = vigenere_decrypt(ciphertext, key_guess)
print(plaintext_candidate[:1000])


## 5. Conclusiones <a id="conclusiones"></a>

En este cuaderno se ha:

1. Implementado el **cifrado y descifrado de Vigenère** en Python.
2. Construido un pequeño **laboratorio de criptoanálisis** que incluye:
   - Cálculo del índice de coincidencia y aplicación del método de Friedman.
   - Análisis de Kasiski para estimar la longitud de la clave.
   - Ruptura de cada columna como un cifrado de César empleando frecuencias de letras.
3. Aplicado las herramientas anteriores a un **criptograma real** propuesto en las tareas
   de SPSI, permitiendo:
   - Estimar la longitud de la clave.
   - Obtener una clave candidata.
   - Aproximarse al texto en claro.

Este enfoque reproduce las ideas clásicas de la criptoanálisis manual, pero apoyadas
en herramientas computacionales que facilitan la experimentación y el análisis
sobre criptogramas más largos.
